In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from datetime import datetime
import time,re
import pandas as pd
import chromedriver_binary
import random
def random_num(max_num):
    return random.randint(2,max_num)

In [6]:
username="########"
keypass="########"

In [23]:
#DBログイン
browser = webdriver.Chrome()

browser.get("https://app.musubu.in/list/companies")
browser.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[1]/form/p[1]/input').send_keys(username)
browser.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[1]/form/p[2]/input').send_keys(keypass)
browser.find_element_by_xpath('//*[@id="loginBtn"]').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="modal-root"]/div[1]/div/div/div[1]').click()
time.sleep(2)


#要検索企業リストを導入
df = pd.read_excel('C:/Users/tong/Desktop/5g/OPPINPUT.xlsx')
database=[]

#要検索企業リストにしたがって検索開始
for index, row in df.iterrows():
    cname=row["企業名"]
    keyword=row["企業名"].translate(str.maketrans({chr(0x0021 + i): chr(0xFF01 + i) for i in range(94)}))
    no=row.No
    
    #検索ボックスをクリアしてからの検索
    browser.find_element_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[3]/input[1]').clear()
    browser.find_element_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[3]/input[1]').send_keys(keyword)
    time.sleep(2)

    companies_name=browser.find_elements_by_xpath('//*[@class="searchListItem__head__info__heading_company_name"]')
    companies_keyword=browser.find_elements_by_xpath('//*[@class="searchListItem__keywords_right"]')
    companies_describe=browser.find_elements_by_xpath('//*[@class="searchListItem__desc"]')
    companies_address=browser.find_elements_by_xpath('//*[@class="searchListItem__head__info__lower flex"]')
    staff=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[1]/div[1]/div[2]')
    year=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[1]/div[2]/div[2]')
    money=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[1]/div[3]/div[2]')                                                                                            
    area=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[2]/div[1]/div[2]')
    area_sub=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[2]/div[2]/div[2]')
    
    #企業名一致性判断
    if keyword in [a.text for a in companies_name]:
        for a,b,c,d,i_staff,i_year,i_money,i_area,i_area_sub in zip(companies_name,companies_keyword,companies_describe,companies_address,staff,year,money,area,area_sub):
            if a.text==keyword:
                b_clean="｜".join(re.findall(r'(?<=name=").+?(?="\>)', b.get_attribute("innerHTML")))
                area_clean="".join(re.findall(r'(?<=cell__).+?(?=</p>)', i_area.get_attribute("innerHTML"))).replace("smlcat\">"," 〇分業界：").replace("bgcat\">","●業界：").replace("&nbsp;,&nbsp;"," ")
                area_sub_clean="".join(re.findall(r'(?<=cell__).+?(?=</p>)',i_area_sub.get_attribute("innerHTML"))).replace("smlcat\">"," 〇分業界：").replace("bgcat\">","●業界：").replace("&nbsp;,&nbsp;"," ")
                database.append([no,cname,a.text,b_clean,c.text,d.text,i_staff.text,i_year.text,i_money.text,area_clean,area_sub_clean])
            else:
                pass    
        print("〇 ",datetime.now()," ",no,".",keyword)
    else:
        database.append([no,keyword,"-","-","-","-","-","-","-","-","-"])
        print("× ",datetime.now()," ",no,".",keyword)
    time.sleep(random_num(4))
browser.close()

df_out = pd.DataFrame(database,columns=["OPPNo","OPP企業","DB会社名","(3)事業内容キーワード","(1)要約（2)概要","(23)住所","(11)従業員数","(6)設立年月","(8)資本金","(4)メイン業界","(5)サブ業界"])
df_out.to_excel('sample.xlsx', sheet_name='sheet1')
df_out

〇  2020-06-23 14:23:45.686875   1 . アーキヤマデ株式会社
〇  2020-06-23 14:23:52.668590   2 . アーク株式会社
〇  2020-06-23 14:23:58.790061   3 . 株式会社アークテック
〇  2020-06-23 14:24:05.170395   4 . 株式会社アースクリエイティブ
×  2020-06-23 14:24:10.444975   5 . アーチャーマインドジャパン株式会社
〇  2020-06-23 14:24:16.878579   6 . 株式会社アート電子
×  2020-06-23 14:24:23.082634   7 . 株式会社アーリーワークス
〇  2020-06-23 14:24:29.533914   8 . 株式会社ＲＲ
〇  2020-06-23 14:24:34.062437   9 . 株式会社アール・ケア
〇  2020-06-23 14:24:38.526277   10 . ＲＫＢ毎日放送株式会社
〇  2020-06-23 14:24:43.882950   11 . ＲＫＢミューズ株式会社
〇  2020-06-23 14:24:49.176121   12 . ＲＳＵＰＰＯＲＴ株式会社
×  2020-06-23 14:24:54.372465   13 . ＲＣＴ株式会社
〇  2020-06-23 14:25:00.874544   14 . 株式会社ＩＩＪグローバルソリューションズ
×  2020-06-23 14:25:05.149363   15 . アイウェルネス株式会社
〇  2020-06-23 14:25:10.452542   16 . 株式会社ＩＨＩ
×  2020-06-23 14:25:16.682735   17 . 株式会社アイエステー
×  2020-06-23 14:25:21.978237   18 . ＩＮＩＳＯＦＴ ＪＡＰＡＮ合同会社
×  2020-06-23 14:25:27.832289   19 . 株式会社ｉＮＤ
〇  2020-06-23 14:25:34.707116   20 . 株式会社アイエムエス
〇  2020-06-23 14:25:40.564592  

〇  2020-06-23 14:39:27.699140   168 . 株式会社アミノ
〇  2020-06-23 14:39:34.468157   169 . 株式会社アミューズ
×  2020-06-23 14:39:39.699331   170 . 有限会社アムザ
〇  2020-06-23 14:39:46.026541   171 . アライ電機産業株式会社
〇  2020-06-23 14:39:52.483360   172 . アライドテレシス株式会社
〇  2020-06-23 14:39:58.798957   173 . 荒谷オート株式会社
〇  2020-06-23 14:40:03.107577   174 . 株式会社アラビア産業
×  2020-06-23 14:40:08.297061   175 . 株式会社アリス・アド
×  2020-06-23 14:40:14.564795   176 . アリスタネットワークスジャパン合同会社
×  2020-06-23 14:40:19.767001   177 . アリティーヴィー株式会社
〇  2020-06-23 14:40:25.100056   178 . アルインコ株式会社
×  2020-06-23 14:40:29.324479   179 . アールエフネットワーク株式会社
〇  2020-06-23 14:40:37.509346   180 . 株式会社アルカディア
〇  2020-06-23 14:40:42.187574   181 . 株式会社アルゴシステム
〇  2020-06-23 14:40:46.515913   182 . ＡＬＳＯＫ大阪株式会社
〇  2020-06-23 14:40:51.882083   183 . ＡＬＳＯＫ近畿株式会社
〇  2020-06-23 14:40:58.207922   184 . ＡＬＳＯＫ福島株式会社
〇  2020-06-23 14:41:02.534300   185 . ＡＬＳＯＫ北海道株式会社
〇  2020-06-23 14:41:08.876616   186 . 株式会社アルタワールド通信社
〇  2020-06-23 14:41:13.232538   187 . 株式会社アルチザネット

〇  2020-06-23 14:54:38.623915   333 . 株式会社ウェザーニューズ
〇  2020-06-23 14:54:43.967445   334 . 株式会社上野原ブロードバンドコミュニケーションズ
〇  2020-06-23 14:54:50.260912   335 . ウエブテック株式会社
〇  2020-06-23 14:54:55.445534   336 . 株式会社ウェルビー
×  2020-06-23 14:55:01.668693   337 . 株式会社ウォークインサイト
×  2020-06-23 14:55:05.852186   338 . 株式会社ウオロクホールディングス
〇  2020-06-23 14:55:11.115582   339 . 株式会社牛木組
〇  2020-06-23 14:55:15.561923   340 . 株式会社内田洋行
×  2020-06-23 14:55:19.826964   341 . 国立研究開発法人宇宙航空研究開発機構
〇  2020-06-23 14:55:25.314865   342 . 株式会社ウッドワン
〇  2020-06-23 14:55:31.621816   343 . 宇部興産コンサルタント株式会社
×  2020-06-23 14:55:37.775943   344 . 宇部市（山口県）
〇  2020-06-23 14:55:44.071777   345 . 株式会社梅里物流サービス
〇  2020-06-23 14:55:50.492542   346 . 梅田運輸倉庫株式会社
×  2020-06-23 14:55:56.664586   347 . うるま市役所（沖縄県）
×  2020-06-23 14:56:01.816141   348 . 雲南市（島根県）
〇  2020-06-23 14:56:06.082355   349 . エアロセンス株式会社
〇  2020-06-23 14:56:12.373380   350 . 株式会社エイアンドシー
〇  2020-06-23 14:56:18.647331   351 . 株式会社エイクラ通信
×  2020-06-23 14:56:23.826372   352 . 

〇  2020-06-23 15:09:02.076987   491 . 株式会社エフエム栃木
〇  2020-06-23 15:09:08.383702   492 . 株式会社エフエム北海道
×  2020-06-23 15:09:12.579434   493 . 株式会社ＦＭよみたん
〇  2020-06-23 15:09:17.192554   494 . 株式会社エフコム
〇  2020-06-23 15:09:24.102962   495 . 株式会社エフ・シー・エス
×  2020-06-23 15:09:30.449201   496 . 株式会社エフ・ジェイ エンターテインメントワークス
×  2020-06-23 15:09:36.740698   497 . Ｆ５ネットワークスジャパン合同会社
〇  2020-06-23 15:09:42.057420   498 . 株式会社エブリスタ
〇  2020-06-23 15:09:48.414809   499 . 株式会社エブリプラン
×  2020-06-23 15:09:54.712335   500 . 株式会社ＥＭＵＲＧＯ Ｊａｐａｎ
×  2020-06-23 15:09:59.462040   501 . 株式会社ＭＩ
×  2020-06-23 15:10:03.777523   502 . 株式会社ＭＩＦモバイル
×  2020-06-23 15:10:08.971208   503 . 株式会社エム＆エム
×  2020-06-23 15:10:15.225737   504 . ＭＨＩ ファシリティーサービス株式会社
〇  2020-06-23 15:10:22.405215   505 . 株式会社エムエスケイ
〇  2020-06-23 15:10:26.761423   506 . ＭＸモバイリング株式会社
〇  2020-06-23 15:10:31.181131   507 . 株式会社エム・エフ・ケイ
〇  2020-06-23 15:10:38.246748   508 . 株式会社エム・エム
〇  2020-06-23 15:10:43.673634   509 . エムオーテックス株式会社
×  2020-06-23 15:10:47.885994  

〇  2020-06-23 15:23:34.223220   654 . 株式会社かがし屋
〇  2020-06-23 15:23:38.506575   655 . 株式会社加賀田組
〇  2020-06-23 15:23:42.853745   656 . 加賀電子株式会社
×  2020-06-23 15:23:48.039808   657 . 各務原市（岐阜県）
〇  2020-06-23 15:23:54.007196   658 . 株式会社加賀屋
×  2020-06-23 15:23:58.532709   659 . 香川県
〇  2020-06-23 15:24:03.560692   660 . 株式会社カキヌマ
〇  2020-06-23 15:24:09.925795   661 . 株式会社柿本商会
〇  2020-06-23 15:24:15.278080   662 . 株式会社カクヤス
×  2020-06-23 15:24:19.495111   663 . 隂山建設株式会社
〇  2020-06-23 15:24:23.882975   664 . 鹿児島テレビ放送株式会社
〇  2020-06-23 15:24:30.215647   665 . 株式会社鹿児島放送
〇  2020-06-23 15:24:36.594952   666 . 株式会社鹿児島讀賣テレビ
〇  2020-06-23 15:24:40.876304   667 . 篭野通信株式会社
〇  2020-06-23 15:24:47.241657   668 . 株式会社河西精機製作所
〇  2020-06-23 15:24:51.980899   669 . カシオ計算機株式会社
×  2020-06-23 15:24:56.259990   670 . 株式会社鹿島アントラーズＦＣ
〇  2020-06-23 15:25:03.223023   671 . 鹿島建設株式会社
〇  2020-06-23 15:25:08.636969   672 . 株式会社菓匠三全
〇  2020-06-23 15:25:14.932695   673 . 株式会社ガゾウ
〇  2020-06-23 15:25:20.284479   674 . カタギリ・コーポ

...

〇  2020-06-23 18:06:08.125259   2436 . 株式会社ＰＬＡＮＴ
〇  2020-06-23 18:06:14.773373   2437 . 株式会社フリーエージェント
〇  2020-06-23 18:06:21.282942   2438 . フリービット株式会社
〇  2020-06-23 18:06:26.001570   2439 . プリマハム株式会社
〇  2020-06-23 18:06:31.579908   2440 . フリュー株式会社
〇  2020-06-23 18:06:36.078080   2441 . プリンス電機株式会社
〇  2020-06-23 18:06:42.536541   2442 . 株式会社ブルーストーンリンクアンドサークル
〇  2020-06-23 18:06:48.165847   2443 . 古河電気工業株式会社
〇  2020-06-23 18:06:52.535086   2444 . 株式会社フルステージ
〇  2020-06-23 18:06:56.959421   2445 . 株式会社フルネス
〇  2020-06-23 18:07:01.327931   2446 . 株式会社ブルボン
〇  2020-06-23 18:07:06.685205   2447 . 株式会社ブレイディア
〇  2020-06-23 18:07:13.200548   2448 . 株式会社プレステージ・インターナショナル
〇  2020-06-23 18:07:19.665110   2449 . 株式会社プロアシスト
×  2020-06-23 18:07:25.014938   2450 . 特定非営利活動法人ブロードバンド・アソシエーション
×  2020-06-23 18:07:30.273697   2451 . Ｂｌｏｃｋｓａｆｅ Ｉｎｃ．
〇  2020-06-23 18:07:35.712050   2452 . 株式会社ブロードバンドタワー
〇  2020-06-23 18:07:40.162526   2453 . ブロードメディアＧＣ株式会社
〇  2020-06-23 18:07:46.559500   2454 . 株式会社プロビズモ
〇  2020-

〇  2020-06-23 18:21:08.673399   2599 . 三菱ＵＦＪモルガン・スタンレー証券株式会社
×  2020-06-23 18:21:13.514690   2600 . ｃ株式会社
〇  2020-06-23 18:21:18.062104   2601 . 株式会社ミップ
〇  2020-06-23 18:21:23.523981   2602 . 株式会社ミツワ
×  2020-06-23 18:21:28.803500   2603 . 水戸市（茨城県）
〇  2020-06-23 18:21:33.251208   2604 . 株式会社ミトリ
〇  2020-06-23 18:21:39.739486   2605 . 港産業株式会社
×  2020-06-23 18:21:43.945785   2606 . 南会津町（島根県）
×  2020-06-23 18:21:49.086234   2607 . 南信州広域連合
×  2020-06-23 18:21:53.261235   2608 . 美波町（徳島県）
〇  2020-06-23 18:21:59.572789   2609 . 株式会社南日本新聞社
〇  2020-06-23 18:22:04.864739   2610 . 株式会社南日本放送
〇  2020-06-23 18:22:10.161613   2611 . ミニストップ株式会社
〇  2020-06-23 18:22:15.618054   2612 . 株式会社みねた
×  2020-06-23 18:22:21.795945   2613 . 学校法人箕面自由学園
×  2020-06-23 18:22:26.949807   2614 . 身延町（山梨県）
〇  2020-06-23 18:22:32.510146   2615 . みのる産業株式会社
×  2020-06-23 18:22:37.677079   2616 . 美浜町（愛知県）
〇  2020-06-23 18:22:43.945733   2617 . ミハル通信株式会社
〇  2020-06-23 18:22:48.216242   2618 . 有限会社三平商会
×  2020-06-23 18:22:54.370

〇  2020-06-23 18:36:07.979718   2763 . 横河レンタ・リース株式会社
〇  2020-06-23 18:36:14.928155   2764 . 株式会社横建
×  2020-06-23 18:36:20.125463   2765 . 横須賀市（神奈川県）
〇  2020-06-23 18:36:24.541193   2766 . 横田工業株式会社
〇  2020-06-23 18:36:30.011725   2767 . 横浜エフエム放送株式会社
〇  2020-06-23 18:36:36.456980   2768 . 株式会社横浜銀行
×  2020-06-23 18:36:40.620872   2769 . 横浜市経済局
〇  2020-06-23 18:36:45.971195   2770 . 株式会社ヨコハマテレナード
×  2020-06-23 18:36:51.120912   2771 . 横浜東邦病院
〇  2020-06-23 18:36:58.320557   2772 . 株式会社ヨシカワ
〇  2020-06-23 18:37:02.633843   2773 . 株式会社吉田システム
〇  2020-06-23 18:37:07.520925   2774 . 株式会社吉田鉄工所
〇  2020-06-23 18:37:12.798483   2775 . 吉田電話株式会社
〇  2020-06-23 18:37:17.096277   2776 . 株式会社吉田東光
×  2020-06-23 18:37:22.337539   2777 . 吉富町（福岡県）
〇  2020-06-23 18:37:28.693501   2778 . 吉乃川株式会社
×  2020-06-23 18:37:33.889647   2779 . 吉野川市（徳島県）
×  2020-06-23 18:37:38.068235   2780 . 学校法人淀之水学院
×  2020-06-23 18:37:42.294044   2781 . 与那国町役場（沖縄県）
×  2020-06-23 18:37:46.457078   2782 . 米子市（鳥取県）
×  2020-06-23 18:37:51.

×  2020-06-23 18:50:13.650138   2921 . Ｃ－Ｔｅｃｈ Ｓｏｌｕｔｉｏｎｓ
×  2020-06-23 18:50:17.850280   2922 . ＣｙｂｅｒＡｇｅｎｔ， Ｉｎｃ．
×  2020-06-23 18:50:24.060463   2923 . Ｄａｙｓ Ｉｎｎ ｂｙ Ｗｙｎｄｈａｍ， Ｇｕａｍ
×  2020-06-23 18:50:29.270304   2924 . ＤＦＳ Ｇｕａｍ ＬＰ
×  2020-06-23 18:50:33.435454   2925 . ＤＯＣＯＭＯ ＰＡＣＩＦＩＣ
×  2020-06-23 18:50:37.640010   2926 . Ｅｄｇｅｍｅｓｈ Ｃｏｒｐｏｒａｔｉｏｎ
×  2020-06-23 18:50:42.930128   2927 . Ｆａｒ ＥａｓＴｏｎｅ Ｔｅｌｅｃｏｍｍｕｎｉｃａｔｉｏｎｓ Ｃｏ．，Ｌｔｄ
×  2020-06-23 18:50:48.130335   2928 . Ｆｉｅｓｔａ Ｒｅｓｏｒｔ Ｇｕａｍ
×  2020-06-23 18:50:52.300083   2929 . Ｆｌａｓｈｆｒａｍｅ ＬＬＣ
×  2020-06-23 18:50:56.508531   2930 . ＦＬＡＴＯＡＫ ＣＯ．， ＬＴＤ．
×  2020-06-23 18:51:01.700347   2931 . Ｇｒａｎｄ Ｐｌａｚａ Ｈｏｔｅｌ
×  2020-06-23 18:51:05.900086   2932 . Ｇｕａｍ Ｆａｓｔ Ｆｏｏｄｓ， ＩＮＣ．
×  2020-06-23 18:51:12.109983   2933 . Ｇｕａｍ Ｐｏｗｅｒ Ａｕｔｈｏｒｉｔｙ
×  2020-06-23 18:51:17.350106   2934 . Ｇｕａｍ Ｙａｍａｎｏｉ， Ｉｎｃ
×  2020-06-23 18:51:23.599641   2935 . Ｇｕａｍ＇ｓ Ｂａｋｅｒｙ， Ｉｎｃ．
×  2020-06-23 18:51:28.769961   2936 . Ｈａｃｋｒｏｄ Ｉｎｃ．
×  2020-06-23 18:51:34.950305   2937 . Ｈｅｘｅｌ Ｗｏｒｋｓ，Ｉｎｃ．
×  2020-0

,OPPNo,OPP企業,DB会社名,キーワード,概要,アドレス,従業員数,設立年月,資本金,メイン業界,サブ業界
0,1,アーキヤマデ株式会社,アーキヤマデ株式会社,合成樹脂製品製造｜プラスチック・樹脂製シート販売｜建築資材販売｜プラスチック・樹脂製シート製...,合成樹脂の建築資材や原料、薬品及び金属製建築資材の販売等を行う会社\n屋根の防水シートを始め...,大阪府吹田市江の木町２４番１０号,149人,1956年3月,8500万円,●業界：建設・工事 〇分業界：樹脂系建材製造,●業界：建設・工事 〇分業界：金属系建材製造 〇分業界： 太陽光パネル
1,2,アーク株式会社,アーク株式会社,太陽光発電｜スマホ・携帯電話販売｜賃貸家賃保証｜不動産情報サイト運営,賃貸家賃保証事業および携帯電話の代理販売などを行う会社\n家賃保証システム「マイガード」の運...,岩手県盛岡市中ノ橋通２丁目８番２号,200人,1992年2月,8000万円,●業界：通信 〇分業界：携帯・通信回線販売代理店,●業界：IT 〇分業界：Webサービス・アプリ運営●業界：エネルギー 〇分業界：再生可能エネルギー
2,3,株式会社アークテック,株式会社アークテック,食品輸送｜トラック運送,食品や資材などの配送を行う会社\n1tから4tまでの車両を用いて、生鮮食品や梱包用資材、さら...,北海道札幌市豊平区月寒東三条４丁目５番２号,9人,-,-,●業界：運輸・物流 〇分業界：一般貨物輸送,
3,3,株式会社アークテック,株式会社アークテック,データ管理システム開発｜セミナー運営｜金融機関向けシステム開発｜電子機器製造｜電子機器販売｜...,人間ドックなどの健診データ収集をするシステムの開発・販売を行う会社\n全国の人間ドックや健診...,東京都中央区日本橋３丁目５番１３号三義ビル,-,1996年7月,4500万円,●業界：IT 〇分業界：ITインフラ,●業界：コンサルティング 〇分業界：ITコンサルティング ●業界：その他サービス 〇分業界：...
4,3,株式会社アークテック,株式会社アークテック,タイル工事｜レンガ販売,建築仕上げ材や家具、照明の販売を行っている会社\n主に建築現場における仕上げ材、レンガやタイ...,東京都品川区上大崎３丁目１０番６０号,-,1998年4月,1000万円,●業界：商社 〇分業界：その他専門商社,●業界：電気製品 〇分業界：照明器具●業界：コンサルティング 〇分業界：土木・建築コンサルテ...
5,4,株式会社アースクリエイティブ,株式会社アースクリエイティブ,浄化槽保守・メンテナンス｜仮設資材レンタル・リース｜浄化槽清掃｜一般廃棄物収集運搬｜プラスチ...,浄化槽の管理や食品や廃プラスチックなどのリサイクル事業を行う会社\nバキュームによる槽内の汚...,山口県宇部市大字妻崎開作１３１９番地の１,-,1979年10月,4000万円,●業界：その他サービス 〇分業界：その他清掃,●業界：小売 〇分業界：キッズアパレルショップ 〇分業界： 古本・リサイクルショップ●業界：...
6,5,アーチャーマインドジャパン株式会社,-,-,-,-,-,-,-,-,-
7,6,株式会社アート電子,株式会社アート電子,プリント基板設計｜ハーネス加工｜プリント基板卸売｜プリント基板製造,大阪・神奈川でプリント基板の設計・製造、電子部品実装・集約を手がける会社\nプリント基板の設...,大阪府堺市西区鳳中町１０丁１６番地１５,-,1974年2月,1000万円,●業界：機械 〇分業界：電子部品製造,
8,6,株式会社アート電子,株式会社アート電子,制御盤製造｜プリント基板製造｜プリント基板設計,システム制御装置や検査装置の設計および製造などを行う会社\nプリント基板を用いたシステム制御...,山口県周南市桜木１丁目３番３０号,-,1986年9月,1000万円,●業界：機械 〇分業界：その他機械製造,●業界：機械関連サービス 〇分業界：機械修理
9,7,株式会社アーリーワークス,-,-,-,-,-,-,-,-,-


In [4]:
#DBログイン
browser = webdriver.Chrome()




#要検索企業リストを導入
df = pd.read_excel('by_keyword.xlsx')
database=[]

for index, row in df.iterrows():
    cname=row["DB会社名"]
    keyword=row["DB会社名"].translate(str.maketrans({chr(0x0021 + i): chr(0xFF01 + i) for i in range(94)}))


,Unnamed: 0,DB会社名,(1)要約,（2)概要,(3)事業内容キーワード,(4)メイン業界,(5)サブ業界,(6)設立年月,(8)資本金,(11)従業員数,(23)住所
0,0,ソフトバンク株式会社,インターネット関連および情報通信を行う会社,携帯端末の販売や、データ通信などの固定通信サービスを行う｢国内通信事業｣を行っている。また、...,決済代行｜スマホ・携帯電話卸売｜データ通信サービス提供｜スマホ・携帯電話販売,●業界：通信 〇分業界：通信回線,●業界：金融 〇分業界：クレジット・信販・決済代行,1986年12月,2043億900万円,1万7115人,東京都港区東新橋１丁目９番１号
1,1,株式会社エヌ・ティ・ティ・データ,官公庁などを対象としたシステムの開発および運用を行う会社,中央省庁や地方自治体を対象に税務や雇用および労働、貿易、交通などの情報システムの開発、運用を...,データ通信サービス提供｜クラウドサービス提供,●業界：IT 〇分業界：システム開発,●業界：IT 〇分業界：クラウド・フィンテック,1988年5月,1425億2000万円,1万1310人,東京都江東区豊洲３丁目３番３号
2,2,ＫＤＤＩ株式会社,個人向けの通信サービスや金融決済ならびにエンターテインメント等のサービスを提供する会社,通信・金融などのサービスを提供しており、パーソナル事業では、au・MVNO携帯電話、FTTH...,クラウドサービス提供｜データ通信サービス提供,●業界：通信 〇分業界：通信回線,●業界：IT 〇分業界：クラウド・フィンテック,1984年6月,1418億5200万円,1万968人,東京都新宿区西新宿２丁目３番２号
3,3,凸版印刷株式会社,印刷技術を基盤とした商品の製造および販売を行う会社,情報コミュニケーション事業として、通帳やＩＣカードなどのセキュア関連、カタログおよびパンフレ...,フィルム製造｜タイル製造｜翻訳ソフト開発｜電子決済システム開発｜ARアプリ開発｜パッケージデ...,●業界：広告 〇分業界：紙媒体印刷,●業界：IT 〇分業界：クラウド・フィンテック 〇分業界： その他IT 〇分業界： システム...,1900年,1049億8600万円,9993人,東京都台東区台東１丁目５番１号
4,4,ＮＥＣプラットフォームズ株式会社,ICTシステム機器の開発や製造や販売、設置および保守などを手掛ける会社,キーテレフォンやルーター等のICTシステム機器の開発や製造や販売、設置および保守などを行う。...,POSシステム開発｜ルーター（置き型）製造｜監視カメラシステム開発｜生産管理システム開発｜医...,●業界：機械 〇分業界：精密機器製造,●業界：IT 〇分業界：システム受託開発 〇分業界： システム開発●業界：コンサルティング ...,1932年11月,103億3100万円,6954人,神奈川県川崎市高津区北見方２丁目６番１号
5,5,エヌ・ティ・ティ・コミュニケーションズ株式会社,ネットワークやデータセンターの開発、構築などを行う会社,企業向けに、クラウドや専用回線を用いたネットワークおよびデータセンターなどの開発ならびに提案...,セキュリティサービス開発｜インターネット回線販売｜IP電話サービス提供｜マーケティング支援｜...,●業界：IT 〇分業界：システム開発,●業界：IT 〇分業界：情報セキュリティサービス 〇分業界： クラウド・フィンテック●業界：...,1999年7月,2309億円,5700人,東京都千代田区大手町2-3-1 大手町プレイスウエストタワー
6,6,コネクシオ株式会社,通信キャリア認定ショップの運営及び通信サービスの契約の取次などを行う会社,通信キャリア認定ショップを運営し、消費者や法人に対して、通信キャリアから供給される携帯電話の...,スマホ・携帯電話販売｜Webメディア運営｜スマホ周辺機器・スマホアクセサリー販売｜データ通信...,●業界：通信 〇分業界：携帯・通信回線販売代理店,●業界：IT 〇分業界：Webサービス・アプリ運営,1997年8月,27億7800万円,5222人,東京都新宿区西新宿８丁目１７番１号
7,7,東日本電信電話株式会社,東日本を対象とした通信サービスの提供を行う会社,東日本を対象に、電話やインターネット回線などといった通信サービスの提供を行う。また、法人の顧...,IP電話サービス提供｜インターネット回線販売｜データ通信サービス提供｜通信機器販売｜ネットワ...,●業界：通信 〇分業界：通信回線,●業界：IT 〇分業界：システム受託開発 〇分業界： ITインフラ,1999年7月,3350億円,4900人,東京都新宿区西新宿３丁目１９番２号
8,8,株式会社ティーガイア,個人や法人向けの通信サービスの契約取次および携帯電話などの販売を手掛ける会社,株式会社NTTドコモやソフトバンク株式会社など通信事業者の一次代理店として、通信サービスの利...,通信回線取次｜インターネット回線販売｜金券・ギフト券（商品券・旅行券・図書カード等）販売｜デ...,●業界：通信 〇分業界：携帯・通信回線販売代理店,●業界：小売 〇分業界：古本・リサイクルショップ●業界：金融 〇分業界：証券,1992年2月,31億5400万円,3725人,東京都渋谷区恵比寿４丁目１番１８号
9,9,グローリー株式会社,貨幣処理機や貨幣端末機、自動販売機などの製造・販売・保守を行う会社,金融機関の営業店において出納管理を行うオープン出納システムや、窓口用紙幣・硬貨入出金機、両替...,自動券売機製造｜両替機卸売｜両替機製造｜監視カメラシステム開発｜機械保守・メンテナンス｜紙幣...,●業界：機械 〇分業界：自動販売機・自動サービス機,●業界：製造 〇分業界：防災・防犯機器●業界：機械関連サービス 〇分業界：機械修理●業界：機...,1944年11月,128億9294万7600円,3447人,兵庫県姫路市下手野１丁目３番１号
